In [1]:
import pandas as pd
import pandas_bokeh
pandas_bokeh.output_notebook()
pd.set_option('plotting.backend', 'pandas_bokeh')
import numpy as np
import sqlite3
import matplotlib
from datetime import date, timedelta
db_conn = sqlite3.connect('cases.d')
total_students = 85000
%load_ext autoreload
%autoreload 2
import techniques
pd.set_option('display.max_rows', None)

Loading BokehJS ...

In [2]:
df_summary = techniques.summarize(db_conn, total_students)
# smooth out the new cases reported on the 21st bc the lump reporting on that day messes with the forecasting
df_smooth = df_summary.copy()
cases = df_smooth.loc[pd.Timestamp(year=2021, month=9, day=21)]['New Student Cases']
cases_per_day = cases / 7.
for d in pd.date_range(start=pd.Timestamp(year=2021, month=9, day=15),
                       end=pd.Timestamp(year=2021, month=9, day=21),
                       freq='D'):
    df = df_smooth.append(pd.Series(name=d, dtype='float64'))
    df_smooth.loc[d, 'New Student Cases'] = cases_per_day
df_smooth = df_smooth.sort_index()

start_date = pd.Timestamp(year=2021, month=9, day=22)

model_high = techniques.Model2(
    df_smooth,
    start_date,
    total_students,
    seroprevalence = 0,
#    r0 = 1.62,
#    r0 = 1.42,
#    r0 = 1.0825,
    r0 = 1.001,
    quarantine_factor = 7.5,
    pct_long_covid = 0.135,
    pct_severe = 0.075,
    pct_death = 0.0001,
    quarantine_period = 10
)

model_low = techniques.Model2(
    df_smooth,
    start_date,
    total_students,
    seroprevalence = 0.25,
#    r0 = 1.62,
#    r0 = 1.42,
#    r0 = 1.0825,
    r0 = 1.001,
    quarantine_factor = 7.5,
    pct_long_covid = 0.135,
    pct_severe = 0.075,
    pct_death = 0.0001,
    quarantine_period = 10
)

for day in pd.date_range(start=start_date, end=pd.Timestamp(year=2021, month=12, day=23), freq='D'):
    model_high.tick()
    model_low.tick()

model_high.df

DatabaseError: Execution failed on sql '
    SELECT * FROM (
      SELECT Datestamp,
             SUM(Active_Student) AS "Active Student Cases",
             SUM(Total_Student) AS "Total Student Cases",
             SUM(Active_Staff) AS "Active Staff Cases",
             SUM(Total_Staff) AS "Total Staff Cases"
      FROM cases
      GROUP BY Datestamp
    ) AS case_summary
    LEFT JOIN (
      SELECT Datestamp,
             Students AS "Quarantined Students",
             Staff AS "Quarantined Staff"
      FROM quarantines
    ) USING(Datestamp);
    ': no such table: cases

In [ ]:
df = pd.DataFrame({
    'Actual Total Cases' : df_summary['Total Student Cases'],
    'Projected Total Cases (High)' : model_high.df['Total Cases'],
    'Projected Total Cases (Low)'  : model_low.df['Total Cases']
})
df.interpolate(method='time', limit_area='inside').plot()

Figure(id='1611', ...)

In [ ]:
df = pd.DataFrame({
    'Actual Quarantine' : df_summary['Quarantined Students'],
    'Projected Quarantine (High)' : model_high.df['Quarantined'],
    'Projected Quarantine (Low)'  : model_low.df['Quarantined']
})
df.interpolate(method='time', limit_area='inside').plot()

Figure(id='1902', ...)